In [9]:
# This is Part B of a project which aims to convert circuit images into editable kicad schematics.
# Part A uses computer vision to create a list of wires and components with their co-ordinates, and their names.
# Part B takes this list and creates a kicad_sch file from it.

# Symbols and components are used synonymously. Examples of symbol/component: resistor, wire, battery, switch, etc

# Kicad_sch file explanation
# (kicad_sch ... ): This is the root element of the schematic file, indicating that it's a KiCad schematic document.
# (version ... ): Specifies the version of the KiCad schematic format.
# (generator "eeschema"): Indicates that the file was generated by 'eeschema', which is KiCad's schematic editor. It can be anything, even "SidYifanOmarNeel"
# (uuid "...."): A universally unique identifier for the schematic.
# (paper "A4"): Defines the paper size for the schematic print layout.
# (lib_symbols ... ): This section contains definitions for all the symbols (components) used in the schematic.
# (wire ... ): Represents a wire connecting different pins in the schematic.
# (symbol ... ): Places an instance of a symbol on the schematic, which references the lib_symbols definition.
# (sheet_instances ... ): Specifies instances of hierarchical sheets within the schematic, if any.
# Notes:
#   kicad_sch file does not store connections between components. On run time, it sees the co-ordinates of the ends of a wire if it is close to the pin of a component then it is considered to be connected.
#   Co-ordinates (x,y):
#       top left corner is (0,0) in kicad_sch and (0,0) in eeschema
#       top right corner is (297.18,0) in kicad_sch and (11700,0) in eeschema
#       bottom left corner is (0,209.55) in kicad_sch and (0,8250) in eeschema
#       bottom right corner is (297.18,209.55) in kicad_sch and (11700,8250) in eeschema
#       Conversion ratio of co-ordinates = eeschema/kicad_sch = 8250/209.55 = 39.3700787402
#       With internal padding, top left corner is (600,600) in eeschema or (15.24,15.24) in kicad_sch

In [10]:
## Input is a list of wires and components, where each of it is a dictionary.
# Ensure Device.kicad_sym and other files are in the same directory as this notebook.
wires = [
    {"x": 148.59, "y": 77.47, "end_x": 157.48, "end_y": 77.47},
    {"x": 138.43, "y": 77.47, "end_x": 133.35, "end_y": 77.47},
    {"x": 133.35, "y": 53.34, "end_x": 142.24, "end_y": 53.34},
    {"x": 133.35, "y": 77.47, "end_x": 133.35, "end_y": 67.31},
    {"x": 133.35, "y": 59.69, "end_x": 133.35, "end_y": 53.34},
    {"x": 157.48, "y": 77.47, "end_x": 157.48, "end_y": 77.47},
    {"x": 157.48, "y": 53.34, "end_x": 149.86, "end_y": 53.34},
] # Keeping it simple for now, can add width to it in the future
components = [
    # {"lib_id": ":SW_DPST_x2", "x": 143.51, "y": 77.47, "angle":0, "reference_name": "SW1A"},
    {"lib_id": "Device:Ammeter_AC", "x": 133.35, "y": 64.77, "angle":0, "reference_name": "BT1"},
    {"lib_id": "Device:Ammeter_AC", "x": 133.35, "y": 64.77, "angle":0, "reference_name": "BT1"},
    {"lib_id": "Device:R", "x": 146.05, "y": 53.34, "angle":90, "reference_name": "R1"}
]

## Helper Functions

In [11]:
import uuid
# generated_uuid = uuid.uuid4()

In [12]:
def create_empty_kicad_sch_template():
    file_uuid = f"{uuid.uuid4()}"
    template = f"""(kicad_sch
    (version 20231120)
    (generator "SidYifanOmarNeel")
    (generator_version "8.0")
    (uuid "{file_uuid}")
    (paper "A4")
    (lib_symbols)
    (sheet_instances
        (path "/"
            (page "1")
        )
    )
)"""
    return template
    
create_empty_kicad_sch_template()

'(kicad_sch\n    (version 20231120)\n    (generator "SidYifanOmarNeel")\n    (generator_version "8.0")\n    (uuid "109ebbc8-9465-4b30-b475-b362ad7fb3df")\n    (paper "A4")\n    (lib_symbols)\n    (sheet_instances\n        (path "/"\n            (page "1")\n        )\n    )\n)'

In [13]:
def extract_subsection(content, subsection):
    # subsection = '(symbol "R"'
    # subsection = '(lib_symbols'
    subsection_start = content.find(subsection)
    if subsection_start == -1:
        return None  # Symbol not found

    balance = 0  # Track the balance of parenthesis
    for i in range(subsection_start, len(content)):
        if content[i] == '(':
            balance += 1
        elif content[i] == ')':
            balance -= 1
        
        if balance == 0:
            subsection_end = i + 1
            return [subsection_start, subsection_end, content[subsection_start:subsection_end]]
    return None
extract_subsection(create_empty_kicad_sch_template(), '(lib_symbols')
extract_subsection(create_empty_kicad_sch_template(), '(sheet_instances')

[188,
 261,
 '(sheet_instances\n        (path "/"\n            (page "1")\n        )\n    )']

In [14]:
# Extracts string which is the definition symbol template from Device.kicad_sym file
def extract_symbol_definition(lib_id):
    lib_name = lib_id.split(":")[0] #Eg: Device
    symbol_name = lib_id.split(":")[1] #Eg: Battery_Cell
    ## Import Library Symbols Definitions
    # This file is the reference which is defines the properties of each components
    path_to_lib_kicad_sym_file = f"{lib_name}.kicad_sym"
    # Read the symbol definition from the device file
    with open(path_to_lib_kicad_sym_file, 'r') as file:
        lib_file_content = file.read()

    symbol_def_string = extract_subsection(lib_file_content, f'(symbol "{symbol_name}"')[2]
    symbol_def_string = symbol_def_string.replace(f'(symbol "{symbol_name}"', f'(symbol "{lib_id}"')
    return symbol_def_string

# Usage: Extract the "R" symbol definition

print(extract_symbol_definition("Device:R_US"))
# print(extract_symbol_definition("Device:Battery_Cell"))

(symbol "Device:R_US"
		(pin_numbers hide)
		(pin_names
			(offset 0)
		)
		(exclude_from_sim no)
		(in_bom yes)
		(on_board yes)
		(property "Reference" "R"
			(at 2.54 0 90)
			(effects
				(font
					(size 1.27 1.27)
				)
			)
		)
		(property "Value" "R_US"
			(at -2.54 0 90)
			(effects
				(font
					(size 1.27 1.27)
				)
			)
		)
		(property "Footprint" ""
			(at 1.016 -0.254 90)
			(effects
				(font
					(size 1.27 1.27)
				)
				(hide yes)
			)
		)
		(property "Datasheet" "~"
			(at 0 0 0)
			(effects
				(font
					(size 1.27 1.27)
				)
				(hide yes)
			)
		)
		(property "Description" "Resistor, US symbol"
			(at 0 0 0)
			(effects
				(font
					(size 1.27 1.27)
				)
				(hide yes)
			)
		)
		(property "ki_keywords" "R res resistor"
			(at 0 0 0)
			(effects
				(font
					(size 1.27 1.27)
				)
				(hide yes)
			)
		)
		(property "ki_fp_filters" "R_*"
			(at 0 0 0)
			(effects
				(font
					(size 1.27 1.27)
				)
				(hide yes)
			)
		)
		(symbol "R_US_0_1"
			(polyl

In [15]:
def extract_property_value(subsection, property_name):
    property_start = subsection.find(f'(property "{property_name}"')
    if property_start == -1:
        return None  # Property not found

    value_start = subsection.find('"', property_start + len(f'(property "{property_name}"')) + 1
    value_end = subsection.find('"', value_start + 1)
    return subsection[value_start + 1:value_end]

# symbol_section = extract_subsection(your_kicad_sch_string, '(symbol "Device:R"')
# if symbol_section:
#     description = extract_property_value(symbol_section[2], "Description")
#     print(description)
# else:
#     print("Symbol section not found")

In [19]:
def create_symbol_string(kicad_sch_file,component_dict):
    
    return template

# {"lib_id": "Device:Battery_Cell", "x": 133.35, "y": 64.77, "angle":0, "reference_name": "BT1"}

In [17]:
## Function
def add_component_to_kicad_sch_file(kicad_sch_file,component_dict):
    ### if symbol for component is not lib_symbol, add it
    libSymbols = extract_subsection(kicad_sch_file, '(lib_symbols')
    if libSymbols[2].find(component_dict["lib_id"]) == -1:
        libSymbols_insert_point = libSymbols[0] + len('(lib_symbols')
        kicad_sch_file = kicad_sch_file[:libSymbols_insert_point] + f"\n {extract_symbol_definition(component_dict['lib_id'])} \n " + kicad_sch_file[libSymbols_insert_point:]

    ### add symbol string
    curr_lib_id = component_dict["lib_id"]

    # get file uuid to add to instance section in the bottom of the file
    uuid_section = extract_subsection(kicad_sch_file, '(uuid')
    if uuid_section:
        # Extract the UUID value
        start = uuid_section[2].find('"') + 1  # Find the first quotation mark
        end = uuid_section[2].find('"', start)  # Find the second quotation mark
        uuid_value = uuid_section[2][start:end]
    
    # get symbol value from lib_symbols
    libSymbols = extract_subsection(kicad_sch_file, '(lib_symbols')
    symbol_section = extract_subsection(libSymbols[2], f'(symbol "{curr_lib_id}"')
    # print(libSymbols[2])
    
    description = extract_property_value(symbol_section[2], "Description")
    
    # get symbol description from lib_symbols
    property_value = extract_property_value(symbol_section[2], "Value")
    
    # TODO: set "at" of each property value = parsed_x + lib_symbol_property_x
    symbol_instance = f"""
(symbol
        (lib_id "{component_dict["lib_id"]}")
        (at {component_dict["x"]} {component_dict["y"]} {component_dict["angle"]})
        (unit 1)
        (exclude_from_sim no)
        (in_bom yes)
        (on_board yes)
        (dnp no)
        (fields_autoplaced yes)
        (uuid "{uuid.uuid4()}")
        (property "Reference" "{component_dict["reference_name"]}"
            (at {component_dict["x"]} {component_dict["y"]} 0)
            (effects
                (font
                    (size 1.27 1.27)
                )
            )
        )
        (property "Value" "{property_value}"
            (at {component_dict["x"]} {component_dict["y"]} 0)
            (effects
                (font
                    (size 1.27 1.27)
                )
            )
        )
        (property "Footprint" ""
            (at {component_dict["x"]} {component_dict["y"]} 0)
            (effects
                (font
                    (size 1.27 1.27)
                )
                (hide yes)
            )
        )
        (property "Datasheet" "~"
            (at {component_dict["x"]} {component_dict["y"]} 0)
            (effects
                (font
                    (size 1.27 1.27)
                )
                (hide yes)
            )
        )
        (property "Description" "{description}"
            (at {component_dict["x"]} {component_dict["y"]} 0)
            (effects
                (font
                    (size 1.27 1.27)
                )
                (hide yes)
            )
        )
        (instances
            (project "temp_{uuid_value}"
                (path "/{uuid_value}"
                    (reference "{component_dict["reference_name"]}")
                    (unit 1)
                )
            )
        )
    )    
"""
    
    
    libSymbols = extract_subsection(kicad_sch_file, '(lib_symbols')
    symbol_insert_point = libSymbols[1] + 1
    # print(f" the value of symbol_insert_point is {symbol_insert_point}")
    kicad_sch_file = kicad_sch_file[:symbol_insert_point] + f"\n {symbol_instance} \n " + kicad_sch_file[symbol_insert_point:]
    # print(kicad_sch_file)
    return kicad_sch_file
add_component_to_kicad_sch_file(create_empty_kicad_sch_template(), {"lib_id": "Device:Battery_Cell", "x": 133.35, "y": 64.77, "angle":0, "reference_name": "BT1"})


TypeError: 'NoneType' object is not subscriptable

In [ ]:
def add_wire_to_kicad_sch_file(kicad_sch_file,wire_dict):
    wire_template = f"""(wire
		(pts
			(xy {wire_dict['x']} {wire_dict['y']}) (xy {wire_dict['end_x']} {wire_dict['end_x']})
		)
		(stroke
			(width 0)
			(type default)
		)
		(uuid "{uuid.uuid4()}")
	)"""
    libSymbols = extract_subsection(kicad_sch_file, '(lib_symbols')
    symbol_insert_point = libSymbols[1] + 1
    kicad_sch_file = kicad_sch_file[:symbol_insert_point] + f"\n {wire_template} \n " + kicad_sch_file[symbol_insert_point:]
    return kicad_sch_file
# {"x": 148.59, "y": 77.47, "end_x": 157.48, "end_y": 77.47}

In [ ]:
# ## Wrapper Function
# def main():
#     # create empty kicad_sch file
#     temp_kicad_sch_file = create_empty_kicad_sch_template()

#     # # add each element to kicad_file
#     for component in components:
#         temp_kicad_sch_file = add_component_to_kicad_sch_file(temp_kicad_sch_file, component)

#     for wire in wires:
#         temp_kicad_sch_file = add_wire_to_kicad_sch_file(temp_kicad_sch_file, wire)
    
#     # save temp file
#     # Specify the path where the file will be saved
#     file_path = f'temp_{uuid.uuid4()}.kicad_sch'
#     with open(file_path, 'w') as file:
#         file.write(temp_kicad_sch_file)
#     print(f"Created file {file_path}")
#     return 
# main()

## Final Function

In [ ]:
## Wrapper Function
def create_kicad_sch_file(wires,components):
    # create empty kicad_sch file
    temp_kicad_sch_file = create_empty_kicad_sch_template()

    # add each element to kicad_file
    for component in components:
        temp_kicad_sch_file = add_component_to_kicad_sch_file(temp_kicad_sch_file, component)

    for wire in wires:
        temp_kicad_sch_file = add_wire_to_kicad_sch_file(temp_kicad_sch_file, wire)
    
    # save temp file
    file_path = f'temp_{uuid.uuid4()}.kicad_sch'
    with open(file_path, 'w') as file:
        file.write(temp_kicad_sch_file)
    print(f"Created file {file_path}")
    return temp_kicad_sch_file

In [ ]:
create_kicad_sch_file(wires,components)